In [8]:
import xmltodict
import json
import pandas as pd
import csv

In [11]:
# XML file name
xml_name ="Doc Gyneco-Nirvana"
with open(xml_name+'.xml', mode='r', encoding='utf-8') as file:
    data = file.read().replace('\n', '')
file.close()
#converts the 
rap_dict = xmltodict.parse(data)

In [12]:
recipx = []
break_list=[]
rhyme=[]
lyrics=[]

measure_count=0
rest=0
verse=1


for measure in rap_dict["score-partwise"]["part"]["measure"]:
    
    measure_count+=1
    recipx.append('='+str(measure_count))
    break_list.append('='+str(measure_count))
    rhyme.append('='+str(measure_count))
    lyrics.append('='+str(measure_count))
        
    #if more than per measure
    if isinstance(measure["note"],list):
        
        for note in measure["note"]:
            if 'rest' in note.keys() or (note['pitch']['step']=='G' and 'lyric' not in note.keys()):
                rest+=1
                
                #Surface
                if 'type' in note.keys():
                    if note["type"]=="32nd":
                        recipx.append(16)
                    elif note["type"]=="16th":
                        recipx.append(8)
                    elif note["type"]=="eighth":
                        recipx.append(4)
                    elif note["type"]=="quarter":
                        recipx.append(2)
                    elif note["type"]=="half":
                        recipx.append(1)
                    elif note["type"]=="whole":
                        recipx.append(0.5)
                    else:
                        recipx.append(0)
                else:
                    recipx.append('pppp')

                break_list.append(0)
                rhyme.append(0)
                lyrics.append('R')
                if 'dot' in note.keys():
                    recipx[len(recipx)-1]=str(recipx[len(recipx)-1])+'.'
            else:
                
                if rest>3:
                    recipx.append('*>Verse'+str(verse))
                    break_list.append('*>Verse'+str(verse))
                    rhyme.append('*>Verse'+str(verse))
                    lyrics.append('*>Verse'+str(verse))
                    verse+=1
                rest=0
                
                if 'chord' in note.keys():
                    
                  
                    
                    #Break1
                    if note['pitch']['step']=='A':            

                        break_list[len(break_list)-1] = 1

                    #Break2 (small break)
                    elif note['pitch']['step']=='B':            

                        break_list[len(break_list)-1] = 2

                    #Rhyme
                    elif note['pitch']['step']=='D':  
                        rhyme[len(rhyme)-1]=1                     

                    
                else:

                    #Surface
                    if note['pitch']['step']=='G':
                        if 'type' in note.keys():
                            if note["type"]=="32nd":
                                recipx.append(16)
                            elif note["type"]=="16th":
                                recipx.append(8)
                            elif note["type"]=="eighth":
                                recipx.append(4)
                            elif note["type"]=="quarter":
                                recipx.append(2)
                            elif note["type"]=="half":
                                recipx.append(1)
                            elif note["type"]=="whole":
                                recipx.append(0.5)
                            else:
                                recipx.append(0)
                        else:
                            recipx.append('kkkk')

                        break_list.append(0)
                        rhyme.append(0)
                        lyrics.append(note['lyric']['text'])
                        if 'dot' in note.keys():
                            recipx[len(recipx)-1]=str(recipx[len(recipx)-1])+'.'
                            


                    #false rest, double the previous note
                    elif note['pitch']['step']=='C':            
                        if not isinstance(recipx[len(recipx)-1],str):                        
                            if 'type' in note.keys():
                                if note["type"]=="32nd":
                                    sil=16
                                elif note["type"]=="16th":
                                    sil=8
                                elif note["type"]=="eighth":
                                    sil=4
                                elif note["type"]=="quarter":
                                    sil=2
                                elif note["type"]=="half":
                                    sil=1
                                elif note["type"]=="whole":
                                    sil=0.5
                                else:
                                    recipx.append(0)
                            else:
                                sil=0
                                
                                
                                
                            if 'dot' in note:
                                recipx[len(recipx)-1]=1/(1/recipx[len(recipx)-1]+1/sil+1/(sil*2))
                            else:           
                                recipx[len(recipx)-1]=1/(1/recipx[len(recipx)-1]+1/sil)
                            
                            
                            
                            
                        else:
                            
                            i=1
                            while isinstance(recipx[len(recipx)-i],str):
                                if '.' in recipx[len(recipx)-1]:
                                    
                                    string=recipx[len(recipx)-1]
                                    dots = string.count('.')
                                    num = float(string.strip('.'))
                                    coef=1
                                    for d in range(dots):
                                        coef += 1/(2**(d+1))
                                    recipx[len(recipx)-1]=float(1/(1/num*coef))
                                    break
                                    
                                if recipx[len(recipx)-1][0]=='=':
                                    rest+=1

                                    #Surface
                                    if 'type' in note.keys():
                                        if note["type"]=="32nd":
                                            recipx.append(16)
                                        elif note["type"]=="16th":
                                            recipx.append(8)
                                        elif note["type"]=="eighth":
                                            recipx.append(4)
                                        elif note["type"]=="quarter":
                                            recipx.append(2)
                                        elif note["type"]=="half":
                                            recipx.append(1)
                                        elif note["type"]=="whole":
                                            recipx.append(0.5)
                                        else:
                                            recipx.append(0)
                                    else:
                                        recipx.append('.')

                                    break_list.append(0)
                                    rhyme.append(0)
                                    lyrics.append('R')
                                    if 'dot' in note.keys():
                                        recipx[len(recipx)-1]=str(recipx[len(recipx)-1])+'.'
                                    i=0
                                    break
                                else:
                                    i+=1
                            if i!=0:
                                if 'type' in note.keys():
                                    if note["type"]=="32nd":
                                        sil=16
                                    elif note["type"]=="16th":
                                        sil=8
                                    elif note["type"]=="eighth":
                                        sil=4
                                    elif note["type"]=="quarter":
                                        sil=2
                                    elif note["type"]=="half":
                                        sil=1
                                    elif note["type"]=="whole":
                                        sil=0.5
                                    else:
                                        recipx.append(0)
                                else:
                                    sil=0
                                if 'dot' in note:
                                    recipx[len(recipx)-1]=1/(1/recipx[len(recipx)-1]+1/sil+1/(sil*2))
                                else:       
                                    recipx[len(recipx)-i]=1/(1/recipx[len(recipx)-i]+1/sil)
                            
                        
        
    #only 1 note per measure
    else:
        
                if 'rest' in measure["note"].keys() or ('lyric' not in measure["note"].keys() and measure["note"]['pitch']['step']=='G'):
                    rest+=1
                    if 'type' in measure["note"].keys():
                        if measure["note"]["type"]=="32nd":
                            recipx.append(16)
                        elif measure["note"]["type"]=="16th":
                            recipx.append(8)
                        elif measure["note"]["type"]=="eighth":
                            recipx.append(4)
                        elif measure["note"]["type"]=="quarter":
                            recipx.append(2)
                        elif measure["note"]["type"]=="half":
                            recipx.append(1)
                        elif measure["note"]["type"]=="whole":
                            recipx.append(0.5)
                        else:
                            recipx.append(0)
                    else:
                        recipx.append(measure["note"]["duration"])
                    break_list.append(0)
                    rhyme.append(0)
                    lyrics.append('R')
                    if 'dot' in measure["note"].keys():
                        recipx[len(recipx)-1]=str(recipx[len(recipx)-1])+'.'
                        
                else:
                    
                    if rest>3:
                        recipx.append('*>Verse'+str(verse))
                        break_list.append('*>Verse'+str(verse))
                        rhyme.append('*>Verse'+str(verse))
                        lyrics.append('*>Verse'+str(verse)) 
                        verse+=1
                    rest=0
                    
                    
        
                    if 'chord' in measure["note"].keys():

                        #Break1
                        if 'pitch' in measure["note"].keys() and measure["note"]['pitch']['step']=='A':            

                            break_list[len(break_list)-1] = 1

                        #Break2 (small break)
                        elif 'pitch' in measure["note"].keys() and measure["note"]['pitch']['step']=='B':            

                            break_list[len(break_list)-1] = 2

                        #Rhyme
                        elif 'pitch' in measure["note"].keys() and measure["note"]['pitch']['step']=='D':            

                            rhyme[len(rhyme)-1] = 1


                    else:

                        #Surface
                        if 'pitch' in measure["note"].keys() and measure["note"]['pitch']['step']=='G':
                            if 'type' in measure["note"].keys():
                                if measure["note"]["type"]=="32nd":
                                    recipx.append(16)
                                elif measure["note"]["type"]=="16th":
                                    recipx.append(8)
                                elif measure["note"]["type"]=="eighth":
                                    recipx.append(4)
                                elif measure["note"]["type"]=="quarter":
                                    recipx.append(2)
                                elif measure["note"]["type"]=="half":
                                    recipx.append(1)
                                elif measure["note"]["type"]=="whole":
                                    recipx.append(0.5)
                                else:
                                    recipx.append(0)
                            else:
                                recipx.append('gggg')

                            break_list.append(0)
                            rhyme.append(0)
                            lyrics.append(measure["note"]['lyric']['text'])
                            if 'dot' in measure["note"].keys():
                                recipx[len(recipx)-1]=str(recipx[len(recipx)-1])+'.'



                        #false rest, double the previous noteA
                        elif 'pitch' in measure["note"].keys() and measure["note"]['pitch']['step']=='C':            
                            if not isinstance(recipx[len(recipx)-1],str):
                                if 'type' in measure["note"].keys():
                                    if measure["note"]["type"]=="32nd":
                                        sil=16
                                    elif measure["note"]["type"]=="16th":
                                        sil=8
                                    elif measure["note"]["type"]=="eighth":
                                        sil=4
                                    elif measure["note"]["type"]=="quarter":
                                        sil=2
                                    elif measure["note"]["type"]=="half":
                                        sil=1
                                    elif measure["note"]["type"]=="whole":
                                        sil=0.5
                                    else:
                                        sil=0
                                else:
                                    recipx.append('aaaa')
                                    
                                if 'dot' in measure["note"].keys():
                                    recipx[len(recipx)-1] = 1/(1/recipx[len(recipx)-1]+1/sil+1/sil**2)
                                else:                                
                                    recipx[len(recipx)-1] = 1/(1/recipx[len(recipx)-1]+1/sil)
                            else:
                                i=1
                                while isinstance(recipx[len(recipx)-i],str):
                                    
                                    if '.' in recipx[len(recipx)-1]:
                                        string=recipx[len(recipx)-1]
                                        dots = string.count('.')
                                        num = float(string.strip('.'))
                                        coef=1
                                        for d in range(dots):
                                            coef += 1/(2**(d+1))

                                        recipx[len(recipx)-1]=float(1/(1/num*coef))
                                        break
                                    
                                    if recipx[len(recipx)-1][0]=='=':
                                        rest+=1

                                        #Surface
                                        if 'type' in measure["note"].keys():
                                            if measure["note"]["type"]=="32nd":
                                                recipx.append(16)
                                            elif measure["note"]["type"]=="16th":
                                                recipx.append(8)
                                            elif measure["note"]["type"]=="eighth":
                                                recipx.append(4)
                                            elif measure["note"]["type"]=="quarter":
                                                recipx.append(2)
                                            elif measure["note"]["type"]=="half":
                                                recipx.append(1)
                                            elif measure["note"]["type"]=="whole":
                                                recipx.append(0.5)
                                            else:
                                                recipx.append(0)
                                        else:
                                            recipx.append('vvvv')

                                        break_list.append(0)
                                        rhyme.append(0)
                                        lyrics.append('R')
                                        if 'dot' in measure["note"].keys():
                                            recipx[len(recipx)-1]=str(recipx[len(recipx)-1])+'.'
                                        i=0
                                        break
                                    else:
                                        i+=1
                                if i!=0:
                                    if 'type' in measure["note"].keys():
                                        if measure["note"]["type"]=="32nd":
                                            sil=16
                                        elif measure["note"]["type"]=="16th":
                                            sil=8
                                        elif measure["note"]["type"]=="eighth":
                                            sil=4
                                        elif measure["note"]["type"]=="quarter":
                                            sil=2
                                        elif measure["note"]["type"]=="half":
                                            sil=1
                                        elif measure["note"]["type"]=="whole":
                                            sil=0.5
                                        else:
                                            recipx.append(0)
                                    else:
                                        sil=0
                                    if 'dot' in measure["note"].keys():
                                        recipx[len(recipx)-1]=1/(1/recipx[len(recipx)-1]+1/sil+1/sil**2)
                                    else:    
                                        recipx[len(recipx)-i]=1/(1/recipx[len(recipx)-i]+1/sil)

for r in range(len(recipx)):
    if isinstance(recipx[r],float):
        if recipx[r]%1 !=0:
            if num==1/(1/1+1/2):
                recipx[r]='1.'
            if num==1/(1/1+1/2+1/4):
                recipx[r]='1..'
            if num==1/(1/1+1/2+1/4+1/8):
                recipx[r]='1...'
            elif (recipx[r] ==1/(1/2+1/4)):
                recipx[r]='2.'
            elif (recipx[r] ==1/(1/2+1/4+1/8)):
                recipx[r]='2..'
            elif (recipx[r] ==1/(1/2+1/4+1/8+1/16)):
                recipx[r]='2...'
            elif recipx[r] ==1/(1/4+1/8):
                recipx[r]='4.'
            elif recipx[r] ==1/(1/4+1/8+1/16):
                recipx[r]='4..'
            elif recipx[r] ==1/(1/4+1/8+1/16+1/32):
                recipx[r]='4...'
            elif recipx[r] ==1/(1/8+1/16):
                recipx[r]='8.'
            elif recipx[r] ==1/(1/8+1/16+1/32):
                recipx[r]='8..'
            elif recipx[r] ==1/(1/8+1/16+1/32+1/64):
                recipx[r]='8...'
            elif recipx[r] ==1/(1/16+1/32):
                recipx[r]='16.'
            elif recipx[r] ==1/(1/16+1/32+1/64):
                recipx[r]='16..'
            elif recipx[r] ==1/(1/16+1/32+1/64+1/128):
                recipx[r]='16...'
            elif recipx[r] ==1/(1/32+1/64):
                recipx[r]='32.'
            elif recipx[r] ==1/(1/32+1/64+1/128):
                recipx[r]='32..'
            elif recipx[r] ==1/(1/32+1/64+1/128+1/256):
                recipx[r]='32...'
        else:
            recipx[r]=int(recipx[r])
            

        
with open(xml_name+'.csv', mode='w',encoding='utf-8') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerows(zip(recipx,break_list,rhyme, lyrics))
f.close()